In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import json
import requests 
from clickhouse_driver import Client

import re

In [2]:
def crawl_json(method, url, params):
    """
    function for crawling
    raise error when http status code is not 200 (success).
    
    @param method: (str)  http method for request
    @param url   : (str)  request url
    @param params: (dict) parameter for request
    """
    if method == 'get':
        res = requests.get(url, params=params)
    elif method == 'post':
        res = requests.post(URL, data=json.dumps(data))
        
    res.raise_for_status()
    
    return res.json()

# API를 활용한 크롤링
* 데이터를 가져온 url은 공개하지 않겠음

In [24]:
with open('config.json') as f:
    json_data = json.load(f)

## 1. 대분류와 중분류 가져오기

`Example`
```python
# sample
list_url = json_data['get_crawldata']['get_category_url']
list_param = {
    'display': 'tree',
    'kind': 'category',
    'lang': 'ko',
    'v': 'v1'
}

crawl_json('get', list_url, list_param)
```

In [29]:
# test['tags'][1]['title']
# test['tags'][1]['id']

# test['tags'][1]['tags'][0]['title']
# test['tags'][1]['tags'][0]['id']
# test['tags'][1]['tags'][0]['converted_tag_for_url']
# test['tags'][1]['tags'][0]['count_job']
# test['tags'][1]['tags'][0]['count_user']

list_url = json_data['get_crawldata']['get_category_url']

list_param = {
    'display': 'tree',
    'kind': 'category',
    'lang': 'ko',
    'v': 'v1'
}

contents = crawl_json('get', list_url, list_param)


main_df = pd.DataFrame()
main_lst = []
_col_lst = ['main_cate', 'main_cate_id', 'main_cate_en', 'medium_cate', 'medium_cate_id', 'medium_cate_en', 'medium_cate_cnt_job', 'medium_cate_cnt_user']
for i in range(len(contents['tags'])):
    
    _main_lst = [[contents['tags'][i]['title'], contents['tags'][i]['id'], contents['tags'][i]['mapping_legacy'], contents['tags'][i]['tags'][j]['title'], contents['tags'][i]['tags'][j]['id'], contents['tags'][i]['tags'][j]['converted_tag_for_url'], contents['tags'][i]['tags'][j]['count_job'], contents['tags'][i]['tags'][j]['count_user']] if contents['tags'][i]['mapping_legacy']!= None else [contents['tags'][i]['title'], contents['tags'][i]['id'], contents['tags'][i]['title'], contents['tags'][i]['tags'][j]['title'], contents['tags'][i]['tags'][j]['id'], contents['tags'][i]['tags'][j]['converted_tag_for_url'], contents['tags'][i]['tags'][j]['count_job'], contents['tags'][i]['tags'][j]['count_user']] for j in range(len(contents['tags'][i]['tags']))]    
#     _main_lst = [test['tags'][i]['title'], test['tags'][i]['id'], test['tags'][i]['mapping_legacy'], test['tags'][i]['tags'][0]['title'], test['tags'][i]['tags'][0]['id'], test['tags'][i]['tags'][0]['converted_tag_for_url'], test['tags'][i]['tags'][0]['count_job'], test['tags'][i]['tags'][0]['count_user']]
        
    _main_df = pd.DataFrame(_main_lst, columns=_col_lst)

    main_df = main_df.append(_main_df, ignore_index=True)

In [30]:
main_df.head()

,main_cate,main_cate_id,main_cate_en,medium_cate,medium_cate_id,medium_cate_en,medium_cate_cnt_job,medium_cate_cnt_user
0,F&B,10057,F&B,식품 MD,761,food md,8,2109
1,F&B,10057,F&B,요리사,748,chef,6,4150
2,F&B,10057,F&B,외식업 종사자,749,restaurant worker,4,5878
3,F&B,10057,F&B,바텐더,747,bartender,2,1004
4,F&B,10057,F&B,외식업 메뉴 개발자,10058,restaurant management,2,1856


In [ ]:
# main_cate_id, medium_cate, medium_cate_id

## 2. 중분류 마다 구인을 하고 있는 회사 리스트 가져오기
관심있는 직군에서만 가져옴

In [12]:
print(main_df['main_cate'].unique())
print(main_df['medium_cate'].unique()[:10])

['F&B' '개발' '건설, 시설' '게임 제작' '경영, 비즈니스' '고객서비스, 리테일' '디자인' '마케팅, 광고'
 '물류, 무역' '미디어' '법률, 법집행기관' '엔지니어링, 설계' '영업' '의료, 제약, 바이오' '인사, 교육'
 '정부, 비영리' '제조, 생산' '금융']
['식품 MD' '요리사' '외식업 종사자' '바텐더' '외식업 메뉴 개발자' '레스토랑 관리자' '영양사' '제과제빵사'
 '서버 개발자' '웹 개발자']


In [31]:
bookmark_cate_lst = ['데이터 엔지니어', '빅데이터 엔지니어', '데이터 사이언티스트', '머신러닝 엔지니어', '서비스 기획자', '사업개발,기획자', '프로젝트 매니저', '전략 기획자', '데이터 분석가']

In [34]:
main_df[main_df['medium_cate'].isin(bookmark_cate_lst)]

,main_cate,main_cate_id,main_cate_en,medium_cate,medium_cate_id,medium_cate_en,medium_cate_cnt_job,medium_cate_cnt_user
15,개발,518,Development,데이터 엔지니어,655,data engineer,198,9375
20,개발,518,Development,데이터 사이언티스트,1024,data scientist,133,6185
21,개발,518,Development,머신러닝 엔지니어,1634,machine learning engineer,133,4626
23,개발,518,Development,빅데이터 엔지니어,1025,big data engineer,112,4874
65,"경영, 비즈니스",507,Business,서비스 기획자,565,service planner,282,15356
66,"경영, 비즈니스",507,Business,"사업개발,기획자",564,business development,255,18484
67,"경영, 비즈니스",507,Business,프로젝트 매니저,559,project manager,228,15684
68,"경영, 비즈니스",507,Business,전략 기획자,563,strategic planner,180,15768
70,"경영, 비즈니스",507,Business,데이터 분석가,656,data analyst,111,7675


In [36]:
json_data['get_crawldata']['get_company_url']

'https://www.wanted.co.kr/api/v4/jobs'

In [37]:
df = main_df[main_df['medium_cate'].isin(bookmark_cate_lst)].copy(deep=True)

In [39]:
import time
start_time = time.time()

##### loop medium_cate #####
list_url = json_data['get_crawldata']['get_company_url']
total_df = pd.DataFrame()

### SET
limit_num = 100

for i in range(len(df)):
    MAIN_CATE_ID = df.iloc[i]['main_cate_id']
    MEDIUM_CATE = df.iloc[i]['medium_cate']
    MEDIUM_CATE_ID = df.iloc[i]['medium_cate_id']
    
    print(MAIN_CATE_ID, MEDIUM_CATE, MEDIUM_CATE_ID)
    
    ### SET 
    learning_state = True
    offset_num = 0
    entire_data = []

    
    while learning_state:

        list_param = {
        'country': 'kr',
        'tag_type_id': str(MEDIUM_CATE_ID),
        'job_sort': 'job.latest_order',
        'years': -1,
        'locations': 'all',
        'offset': offset_num,
        'limit': limit_num}

        list_res = crawl_json('get', list_url, list_param) # data가 있는지 없는지 확인하기 위함

        if list_res['data'] == None: # 데이터가 없으면 learning_state = False 로 주어 다음이 안 돌아가도록 함
            print("====> There is No more offset_num: {}".format(offset_num))
            print('end')
            learning_state = False

        else: # 데이터가 있는 경우는 계속 저장
            print("offset_num: {}".format(offset_num))
    #         print(list_res)
            entire_data += list_res['data']
#             offset_num += 1
            offset_num += limit_num
            
        if (offset_num%120 == 0 and offset_num!=0):
            wait_min = 15
            print("Sleep {}min".format(15/60))
            time.sleep(wait_min/60)
    
    
    # ENTIRE_DATA NEED PARSING
    info_col_name = ['company_name', 'company_id', 'industry_name', 'country', 'location', 'job_id', 'job_position', 'job_like_count']
    list_info = [[i['company']['name'], i['company']['id'], i['company']['industry_name'], i['address']['country'], i['address']['location'], i['id'], i['position'],  i['like_count']] for i in entire_data]
#     print(list_info)
#     print(info_col_name)
    
    
    # set main_cate_id, medium_cate, medium_cate_id
    _total_df = pd.DataFrame(list_info, columns=info_col_name)
    _total_df['main_cate_id'] = MAIN_CATE_ID
    _total_df['medium_cate'] = MEDIUM_CATE
    _total_df['medium_cate_id'] = MEDIUM_CATE_ID
    
    # val_columns 은 Column명 정의
#     print("===> To Clickhouse database")
#     client.execute('INSERT INTO crawl.wanted_job_id ({}) VALUES'.format(val_columns), _total_df.values.tolist())
    
#     print(_total_df.head())
    
    total_df = total_df.append(_total_df, ignore_index=True)

    
elapsed_time = time.time() - start_time
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

518 데이터 엔지니어 655
offset_num: 0
offset_num: 100
====> There is No more offset_num: 200
end
518 데이터 사이언티스트 1024
offset_num: 0
offset_num: 100
====> There is No more offset_num: 200
end
518 머신러닝 엔지니어 1634
offset_num: 0
offset_num: 100
====> There is No more offset_num: 200
end
518 빅데이터 엔지니어 1025
offset_num: 0
offset_num: 100
====> There is No more offset_num: 200
end
507 서비스 기획자 565
offset_num: 0
offset_num: 100
offset_num: 200
====> There is No more offset_num: 300
end
507 사업개발,기획자 564
offset_num: 0
offset_num: 100
offset_num: 200
====> There is No more offset_num: 300
end
507 프로젝트 매니저 559
offset_num: 0
offset_num: 100
offset_num: 200
====> There is No more offset_num: 300
end
507 전략 기획자 563
offset_num: 0
offset_num: 100
====> There is No more offset_num: 200
end
507 데이터 분석가 656
offset_num: 0
====> There is No more offset_num: 100
end
00:01:16


In [43]:
print(total_df.columns)
total_df[['company_id', 'job_id', 'job_position']]

Index(['company_name', 'company_id', 'industry_name', 'country', 'location',
       'job_id', 'job_position', 'job_like_count', 'main_cate_id',
       'medium_cate', 'medium_cate_id'],
      dtype='object')


,company_id,job_id,job_position
0,90,29199,Data Scientist
1,9000,39671,데이터베이스 관리자(DBA)
2,1628,28331,Search Service Backend Engineer
3,3606,36523,DBA(Database Engineer)
4,5866,27569,빅데이터/그래프데이터 분석
...,...,...,...
1460,137,30399,VC전략팀 시니어
1461,886,29182,[인텔리전스랩스] 분석지원팀 데이터 분석가
1462,331,25065,서비스 기획자
1463,2330,23846,Data Privacy Lead


## 3. 각 상세정보 가져오기

In [55]:
job_df = total_df[['company_id', 'company_name', 'job_id', 'job_position', 'medium_cate_id', 'medium_cate', 'location']].copy(deep=True)

In [67]:
def get_jobDesc(i):
    """
    i(int): indes of Dataframe
    """
    detail_url = base_url.format(job_df.iloc[i]['job_id'])
    detail_res = crawl_json('get', detail_url, {}) 
    detail = detail_res['job']['detail']
    
    for k, v in detail.items():
        detail[k] = [re.sub(r'[\t\r\n]', '', s) for s in re.split('[•|-]', detail[k]) if s not in ['']] 
        
    
    company_id = job_df.iloc[i]['company_id']
    company_name = job_df.iloc[i]['company_name']
    job_id = job_df.iloc[i]['job_id']
    job_position = job_df.iloc[i]['job_position']
    medium_cate_id = job_df.iloc[i]['medium_cate_id']
    medium_cate = job_df.iloc[i]['medium_cate']
    location = job_df.iloc[i]['location']
    
    desc_col_list = ['requirements', 'main_tasks', 'preferred_points', 'benefits', 'intro']
    
    desc_dict = {}
    
    for ii in desc_col_list:
        try:
            desc_dict[ii] = detail[ii]
        except KeyError as e:
            desc_dict[ii] = detail.get(e, list("-"))
#             print("{} e :: {} {} {}".format(i, e, detail_url, desc_dict[ii]))
            print("{} e :: {} {}".format(i, e, desc_dict[ii]))
            
    new_data = [company_id, company_name, job_id, job_position, medium_cate_id, medium_cate, location, desc_dict['requirements'], desc_dict['main_tasks'], desc_dict['preferred_points'], desc_dict['benefits'], desc_dict['intro']]
    
    if (i % 150 == 0 and i != 0):
#         print('---->{}'.format(detail_url))
        print('---->{}'.format('getting {} detail_url'.format(i)))
        elapsed_time = time.time() - start_time
        print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))
        time.sleep(10)
    
    
    
    return pd.DataFrame(new_data).T

In [50]:
from clickhouse_driver import Client

client = Client('localhost')

In [51]:
create_database = """
CREATE DATABASE IF NOT EXISTS test
"""
client.execute(create_database)

[]

In [52]:
client.execute("SHOW DATABASES")

[('crawl',), ('default',), ('system',), ('test',)]

In [53]:
# Database에 저장하기 위한 Table을 만든다. 

create_tbl = """
    CREATE TABLE IF NOT EXISTS test.wanted_job_desc (
        `insert_time` DateTime DEFAULT now(),
        `company_id` UInt32,
        `company_name` String, 
        `job_id` UInt32,   
        `job_position` String,
        `medium_cate_id` UInt32, 
        `medium_cate` String,  
        `location` String, 
        `requirements` Array(String),  
        `main_tasks` Array(String),    
        `preferred_points` Array(String),  
        `benefits` Array(String),  
        `intro` Array(String)
    ) ENGINE = MergeTree PARTITION BY medium_cate_id ORDER BY insert_time SETTINGS index_granularity = 8192    
"""

client.execute(create_tbl)

[]

In [54]:
columns = ['company_id',
 'company_name',
 'job_id',
 'job_position',
 'medium_cate_id',
 'medium_cate',
 'location',
 'requirements',
 'main_tasks',
 'preferred_points',
 'benefits',
 'intro']


print('`{}`'.format('`,`'.join(columns)).replace(',', ',\n'))

val_columns = ','.join(columns)
'INSERT INTO test.wanted_job_desc ({}) VALUES'.format(val_columns)

# client.execute('INSERT INTO crawl.wanted_job_desc ({}) VALUES'.format(val_columns), _total_df.values.tolist())

`company_id`,
`company_name`,
`job_id`,
`job_position`,
`medium_cate_id`,
`medium_cate`,
`location`,
`requirements`,
`main_tasks`,
`preferred_points`,
`benefits`,
`intro`


'INSERT INTO test.wanted_job_desc (company_id,company_name,job_id,job_position,medium_cate_id,medium_cate,location,requirements,main_tasks,preferred_points,benefits,intro) VALUES'

In [66]:
# get job detail
# 조금씩 데이터를 설정해서 넣는 방법?? => Batch

base_url = json_data['get_crawldata']['get_desc_url']

import re
import copy 
import numpy as np


start_time = time.time()

### SET ###
# dataframe for batch Insert
# row_num [0:100], [101:200], ....
# While False => insert 


batch_size = 100 # 몇개씩 insert 할 것인가
total_step = len(job_df)//batch_size

_total_df = pd.DataFrame()

for step in range(total_step+1):
    new_df = pd.DataFrame()
    
    if (step+1)*batch_size < len(job_df):
        print(step*batch_size, (step+1)*batch_size)
        
        for i in range(step*batch_size, (step+1)*batch_size):
            _job_df = get_jobDesc(i)
            new_df = new_df.append(_job_df, ignore_index=False)
            
            
    else:
#         print((step)*batch_size, len(job_df)-(step)*batch_size+(step*batch_size))
        print(step*batch_size, len(job_df))
        for i in range(step*batch_size, len(job_df)):
            _job_df = get_jobDesc(i)
            new_df = new_df.append(_job_df, ignore_index=False)
    
    print('{}/{}'.format(step, total_step+1))
#     new_df = new_df.append(new_df, ignore_index=True)
    
    print("===> To Clickhouse database")
#     client.execute('INSERT INTO test.wanted_job_desc ({}) VALUES'.format(val_columns), new_df.values.tolist())                                  
    elapsed_time = time.time() - start_time
    print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

    _total_df = _total_df.append(new_df, ignore_index=True)
elapsed_time = time.time() - start_time
print(time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))



0 100
0/15
===> To Clickhouse database
00:00:42
100 200
---->gettting 150 detail_url
00:01:14
1/15
===> To Clickhouse database
00:01:46
200 300
2/15
===> To Clickhouse database
00:02:27
300 400
---->gettting 300 detail_url
00:02:27
3/15
===> To Clickhouse database
00:03:20
400 500
---->gettting 450 detail_url
00:03:42
4/15
===> To Clickhouse database
00:04:13
500 600
5/15
===> To Clickhouse database
00:05:02
600 700
---->gettting 600 detail_url
00:05:03
6/15
===> To Clickhouse database
00:05:59
700 800


IndexError: tuple index out of range